In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 3
%config IPythonBackend.figure_format = 'retina'

import os
from dataclasses import asdict

from project_config import DataPaths, S3DataPaths
from transformers import AutoTokenizer

from my_code.data import mlm
from my_code.train import get_model

try:
    import rich

    rich.reconfigure(force_terminal=True, force_jupyter=False)
    rich.pretty.install()
    print = rich.get_console().out
    rprint = rich.get_console().print
except:
    pass

datapaths_local = DataPaths()
datapaths_s3 = S3DataPaths()
display(asdict(datapaths_local), asdict(datapaths_s3))


{
    'data_prefix': PosixPath('data'),
    'imgs': PosixPath('data/imgs-clean'),
    'textracted': PosixPath('data/textracted'),
    'annotations': PosixPath('data/annotations')
}

{
    'data_prefix': S3Path('s3://sagemaker-ap-southeast-1-111122223333/textract-transformers/data'),
    'imgs': PosixPath('data/imgs-clean'),
    'textracted': PosixPath('data/textracted'),
    'annotations': PosixPath('data/annotations'),
    'bucket_name': S3Path('s3://sagemaker-ap-southeast-1-111122223333'),
    'bucket_prefix': 'textract-transformers'
}


In [2]:
# Let's figure out the tokenizer to use, by looking at the mlm-pretraining job log.
rprint(
    "ModelArguments(cache_dir='/tmp/transformers/cache', config_name=None, model_name_or_path='microsoft/layoutlm-base-uncased', model_revision='main', tokenizer_name=None, use_auth_token=False)"
)
print()
rprint(
    "DataTrainingArguments(annotation_attr='labels', max_seq_length=512, max_train_samples=None, task_name='mlm', textract='/opt/ml/input/data/textract', textract_prefix='textract-transformers/data/textracted', train='/opt/ml/input/data/train', validation='/opt/ml/input/data/validation', num_labels=2, mlm_probability=0.15)"
)

ModelArguments(cache_dir='/tmp/transformers/cache', config_name=None, 
model_name_or_path='microsoft/layoutlm-base-uncased', model_revision='main', 
tokenizer_name=None, use_auth_token=False)

DataTrainingArguments(annotation_attr='labels', max_seq_length=512, 
max_train_samples=None, task_name='mlm', textract='/opt/ml/input/data/textract',
textract_prefix='textract-transformers/data/textracted', 
train='/opt/ml/input/data/train', validation='/opt/ml/input/data/validation', 
num_labels=2, mlm_probability=0.15)


In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # For notebook only.

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    cache_dir=datapaths_local.data_prefix / "transformers/cache",
    use_fast=True,
    revision="main",
    use_auth_token=False,
)

ds_mlm = mlm.TextractLayoutLMDatasetForLM(
    textract_path=str(datapaths_local.textracted),
    tokenizer=tokenizer,
)

Downloading:   0%|          | 0.00/170 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[WARNING|tokenization_utils_base.py:3325] 2022-01-18 11:06:22,348 >> Token indices sequence length is longer than the specified maximum sequence length for this model (924 > 512). Running this sequence through the model will result in indexing errors


In [5]:
# TODO: GITROOT/notebooks/my_code/data/mlm.py:TextractLayoutLMDatasetForLM